Create the SCD Type 2 Table

In [0]:
# Create the SCD Type 2 table
# This stores full history of customer changes

spark.sql("USE CATALOG ecommerce")

spark.sql("""
    CREATE TABLE IF NOT EXISTS ecommerce.silver.customers_scd (
        -- Natural key
        customer_id               STRING NOT NULL,
        customer_unique_id        STRING,

        -- Tracked columns (changes create new versions)
        customer_city             STRING,
        customer_state            STRING,
        customer_zip_code_prefix  INTEGER,

        -- SCD Type 2 metadata columns
        effective_start_date      TIMESTAMP NOT NULL,
        effective_end_date        TIMESTAMP NOT NULL,
        is_current                BOOLEAN NOT NULL,
        version                   INTEGER NOT NULL,

        -- Audit columns
        updated_at                TIMESTAMP
    )
    USING DELTA
    COMMENT 'SCD Type 2 customer history table.
             Tracks changes in city, state and zip code.
             Each row represents one version of a customer record.
             is_current=TRUE means the latest version.'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true'
    )
""")

print("customers_scd table created!")

# Verify
spark.sql("DESCRIBE TABLE ecommerce.silver.customers_scd").show(truncate=False)

customers_scd table created!
+------------------------+---------+-------+
|col_name                |data_type|comment|
+------------------------+---------+-------+
|customer_id             |string   |NULL   |
|customer_unique_id      |string   |NULL   |
|customer_city           |string   |NULL   |
|customer_state          |string   |NULL   |
|customer_zip_code_prefix|int      |NULL   |
|effective_start_date    |timestamp|NULL   |
|effective_end_date      |timestamp|NULL   |
|is_current              |boolean  |NULL   |
|version                 |int      |NULL   |
|updated_at              |timestamp|NULL   |
+------------------------+---------+-------+



Load Initial Batch (Version 1)

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Define a far future date for active records
FUTURE_DATE = "9999-12-31 00:00:00"

# ── Load initial customers from silver ──────────────────────────
print("Loading initial customer batch (Version 1)...")

initial_customers = spark.sql("""
    SELECT
        customer_id,
        customer_unique_id,
        customer_city,
        customer_state,
        customer_zip_code_prefix
    FROM ecommerce.silver.customers
""")

# Add SCD Type 2 columns for initial load
initial_scd = (
    initial_customers
        .withColumn("effective_start_date",
            F.lit("2016-01-01 00:00:00").cast("timestamp"))
        .withColumn("effective_end_date",
            F.lit(FUTURE_DATE).cast("timestamp"))
        .withColumn("is_current",   F.lit(True))
        .withColumn("version",      F.lit(1))
        .withColumn("updated_at",   F.current_timestamp())
)

# Write initial batch
(initial_scd.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("ecommerce.silver.customers_scd")
)

count = spark.table("ecommerce.silver.customers_scd").count()
print(f"Initial load complete: {count:,} customer records")
print(f"All records: is_current=TRUE, version=1")

Loading initial customer batch (Version 1)...
Initial load complete: 99,441 customer records
All records: is_current=TRUE, version=1


Simulate Customer Changes (Batch 2)

In [0]:
# Simulate 500 customers changing their city/state
# This represents customers who moved or updated profiles

print("Simulating customer changes (Batch 2)...")

# Pick 500 random customers to update
customers_to_update = spark.sql("""
    SELECT customer_id, customer_unique_id
    FROM ecommerce.silver.customers_scd
    WHERE is_current = TRUE
    LIMIT 500
""")

# New cities/states they "moved" to
new_locations = [
    ("Sao Paulo", "SP", 1001),
    ("Rio De Janeiro", "RJ", 2001),
    ("Brasilia", "DF", 3001),
    ("Salvador", "BA", 4001),
    ("Fortaleza", "CE", 5001),
    ("Belo Horizonte", "MG", 6001),
    ("Manaus", "AM", 7001),
    ("Curitiba", "PR", 8001),
    ("Recife", "PE", 9001),
    ("Porto Alegre", "RS", 1101),
]

# Assign random new locations
import random
from pyspark.sql.types import StringType, IntegerType

# Create updated records
updated_data = []
customers_list = customers_to_update.collect()

for i, row in enumerate(customers_list):
    new_loc = new_locations[i % len(new_locations)]
    updated_data.append((
        row.customer_id,
        row.customer_unique_id,
        new_loc[0],  # new city
        new_loc[1],  # new state
        new_loc[2],  # new zip
    ))

# Create DataFrame of changes
schema = ["customer_id", "customer_unique_id",
          "customer_city", "customer_state",
          "customer_zip_code_prefix"]

updates_df = spark.createDataFrame(updated_data, schema)

print(f"Generated {updates_df.count()} customer updates")
print("\nSample changes:")
updates_df.show(5)

Simulating customer changes (Batch 2)...
Generated 500 customer updates

Sample changes:
+--------------------+--------------------+--------------+--------------+------------------------+
|         customer_id|  customer_unique_id| customer_city|customer_state|customer_zip_code_prefix|
+--------------------+--------------------+--------------+--------------+------------------------+
|e3c7e245a96d7fa33...|79051ee5ee98c4bd6...|     Sao Paulo|            SP|                    1001|
|a56b03f5e6015f1a5...|b6cbe1a8674ee23e9...|Rio De Janeiro|            RJ|                    2001|
|d0615859a639a94c1...|9072b46e3b6896156...|      Brasilia|            DF|                    3001|
|c0fe0fbc24994167d...|839bbfd4ff93b592c...|      Salvador|            BA|                    4001|
|5b5f4957a69d537a2...|bb03ed8d9549898e8...|     Fortaleza|            CE|                    5001|
+--------------------+--------------------+--------------+--------------+------------------------+
only showing top 5 r

Apply SCD Type 2 MERGE

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType
)

FUTURE_DATE     = "9999-12-31 00:00:00"
change_time_str = "2018-01-01 00:00:00"

# ================================================================
# PART 1: Generate Customer Changes
# ================================================================
print(" Simulating customer changes (Batch 2)...")

customers_to_update = spark.sql("""
    SELECT customer_id, customer_unique_id
    FROM ecommerce.silver.customers_scd
    WHERE is_current = TRUE
    LIMIT 500
""")

new_locations = [
    ("Sao Paulo",      "SP", 1001),
    ("Rio De Janeiro", "RJ", 2001),
    ("Brasilia",       "DF", 3001),
    ("Salvador",       "BA", 4001),
    ("Fortaleza",      "CE", 5001),
    ("Belo Horizonte", "MG", 6001),
    ("Manaus",         "AM", 7001),
    ("Curitiba",       "PR", 8001),
    ("Recife",         "PE", 9001),
    ("Porto Alegre",   "RS", 1101),
]

updated_data = []
customers_list = customers_to_update.collect()

for i, row in enumerate(customers_list):
    new_loc = new_locations[i % len(new_locations)]
    updated_data.append((
        row.customer_id,
        row.customer_unique_id,
        new_loc[0],
        new_loc[1],
        int(new_loc[2]),  # ← explicitly cast to int
    ))

# ── Define schema with explicit types ────────────────────────────
schema = StructType([
    StructField("customer_id",              StringType(),  True),
    StructField("customer_unique_id",       StringType(),  True),
    StructField("customer_city",            StringType(),  True),
    StructField("customer_state",           StringType(),  True),
    StructField("customer_zip_code_prefix", IntegerType(), True),  # ← INTEGER
])

updates_df = spark.createDataFrame(updated_data, schema)
print(f" Generated {updates_df.count()} customer updates")
print("\nSchema of updates_df:")
updates_df.printSchema()

# ================================================================
# PART 2: SCD Type 2 Merge Function
# ================================================================

def apply_scd2_merge(updates_df, change_time):

    # ── Step 1: Snapshot current records ────────────────────────
    print("\nStep 1: Snapshotting current records...")
    current_snapshot = (
        spark.table("ecommerce.silver.customers_scd")
            .filter(F.col("is_current") == True)
            .cache()
    )
    current_snapshot.count()
    print(f"   Snapshot: {current_snapshot.count():,} records")

    # ── Find changed records ─────────────────────────────────────
    changed_records = (
        updates_df.alias("new")
            .join(
                current_snapshot.alias("old"),
                "customer_id",
                "inner"
            )
            .filter(
                (F.col("new.customer_city")  != F.col("old.customer_city")) |
                (F.col("new.customer_state") != F.col("old.customer_state")) |
                (F.col("new.customer_zip_code_prefix") !=
                 F.col("old.customer_zip_code_prefix"))
            )
    )

    changed_count = changed_records.count()
    print(f"   Found {changed_count} records that changed")

    if changed_count == 0:
        print("   No changes detected — skipping")
        current_snapshot.unpersist()
        return

    # ── Step 2: Close old records ────────────────────────────────
    print("Step 2: Closing old records...")
    scd_table = DeltaTable.forName(
        spark, "ecommerce.silver.customers_scd"
    )

    (scd_table.alias("target")
        .merge(
            updates_df.alias("source"),
            """
            target.customer_id = source.customer_id
            AND target.is_current = TRUE
            AND (
                target.customer_city != source.customer_city
                OR target.customer_state != source.customer_state
                OR target.customer_zip_code_prefix !=
                   source.customer_zip_code_prefix
            )
            """
        )
        .whenMatchedUpdate(set={
            "is_current":         "FALSE",
            "effective_end_date": f"CAST('{change_time}' AS TIMESTAMP)",
            "updated_at":         "current_timestamp()"
        })
        .execute()
    )
    print("    Old records closed")

    # ── Step 3: Insert new records ───────────────────────────────
    print("Step 3: Inserting new records...")

    max_versions = (
        spark.table("ecommerce.silver.customers_scd")
            .groupBy("customer_id")
            .agg(F.max("version").alias("max_version"))
    )

    new_records = (
        changed_records
            .select(
                F.col("new.customer_id"),
                F.col("old.customer_unique_id"),
                F.col("new.customer_city"),
                F.col("new.customer_state"),
                F.col("new.customer_zip_code_prefix")
                 .cast(IntegerType()),              # ← explicit cast
            )
            .join(max_versions, "customer_id", "inner")
            .select(
                F.col("customer_id"),
                F.col("customer_unique_id"),
                F.col("customer_city"),
                F.col("customer_state"),
                F.col("customer_zip_code_prefix"),
                F.lit(change_time).cast("timestamp")
                 .alias("effective_start_date"),
                F.lit(FUTURE_DATE).cast("timestamp")
                 .alias("effective_end_date"),
                F.lit(True).alias("is_current"),
                (F.col("max_version") + 1).alias("version"),
                F.current_timestamp().alias("updated_at")
            )
    )

    new_count = new_records.count()
    print(f"   Inserting {new_count} new records...")

    (new_records.write
        .format("delta")
        .mode("append")
        .saveAsTable("ecommerce.silver.customers_scd")
    )

    print(f"    {new_count} new records inserted")
    current_snapshot.unpersist()

# ================================================================
# PART 3: Run the Merge
# ================================================================
print("\n  Applying SCD Type 2 MERGE...")
apply_scd2_merge(updates_df, change_time_str)
print("\n SCD Type 2 MERGE complete!")


 Simulating customer changes (Batch 2)...
 Generated 500 customer updates

Schema of updates_df:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)


  Applying SCD Type 2 MERGE...

Step 1: Snapshotting current records...
   Snapshot: 98,941 records
   Found 500 records that changed
Step 2: Closing old records...
    Old records closed
Step 3: Inserting new records...
   Inserting 0 new records...
    0 new records inserted

 SCD Type 2 MERGE complete!


Simulate Second Batch of Changes

In [0]:
# Simulate 200 more customers changing (Batch 3)
# Some of these will be customers who already changed once
# This tests that version numbers increment correctly

print(" Simulating second batch of changes (Batch 3)...")

# Pick 200 customers from the ALREADY CHANGED ones
second_updates_list = customers_list[:200]

more_updates = []
for i, row in enumerate(second_updates_list):
    new_loc = new_locations[(i + 3) % len(new_locations)]
    more_updates.append((
        row.customer_id,
        row.customer_unique_id,
        new_loc[0],
        new_loc[1],
        new_loc[2],
    ))

more_updates_df = spark.createDataFrame(more_updates, schema)

# Apply second merge
apply_scd2_merge(more_updates_df, "2019-01-01 00:00:00")
print("\n Second batch of changes applied!")

 Simulating second batch of changes (Batch 3)...

Step 1: Snapshotting current records...
   Snapshot: 98,441 records
   Found 0 records that changed
   No changes detected — skipping

 Second batch of changes applied!


Verify SCD Type 2 is Working

In [0]:
print("=" * 60)
print("SCD TYPE 2 VERIFICATION")
print("=" * 60)

# ── 1. Total records ─────────────────────────────────────────────
total     = spark.table("ecommerce.silver.customers_scd").count()
current   = spark.sql("SELECT COUNT(*) as c FROM ecommerce.silver.customers_scd WHERE is_current = TRUE").collect()[0]["c"]
historical = total - current

print(f"\n📊 Record Counts:")
print(f"   Total records:      {total:,}")
print(f"   Current records:    {current:,}  (is_current=TRUE)")
print(f"   Historical records: {historical:,} (is_current=FALSE)")

# ── 2. Version distribution ──────────────────────────────────────
print(f"\n📊 Version Distribution:")
spark.sql("""
    SELECT
        version,
        COUNT(*) as record_count,
        SUM(CASE WHEN is_current THEN 1 ELSE 0 END) as current_count
    FROM ecommerce.silver.customers_scd
    GROUP BY version
    ORDER BY version
""").show()

# ── 3. Sample customer history ───────────────────────────────────
print(" Sample Customer Full History:")
spark.sql("""
    SELECT
        customer_id,
        customer_city,
        customer_state,
        effective_start_date,
        effective_end_date,
        is_current,
        version
    FROM ecommerce.silver.customers_scd
    WHERE customer_id IN (
        SELECT customer_id
        FROM ecommerce.silver.customers_scd
        GROUP BY customer_id
        HAVING COUNT(*) > 1
        LIMIT 3
    )
    ORDER BY customer_id, version
""").show(20, truncate=False)

# ── 4. Verify no overlapping records ────────────────────────────
print(" Data Quality Check — No overlapping date ranges:")
overlap_check = spark.sql("""
    SELECT COUNT(*) as overlap_count
    FROM ecommerce.silver.customers_scd a
    JOIN ecommerce.silver.customers_scd b
        ON a.customer_id = b.customer_id
        AND a.version != b.version
        AND a.effective_start_date < b.effective_end_date
        AND a.effective_end_date > b.effective_start_date
""").collect()[0]["overlap_count"]

if overlap_check == 0:
    print("    No overlapping date ranges found!")
else:
    print(f"     {overlap_check} overlapping records found!")

# ── 5. Verify only one current record per customer ───────────────
print(" Data Quality Check — One current record per customer:")
duplicate_current = spark.sql("""
    SELECT COUNT(*) as dup_count
    FROM (
        SELECT customer_id, COUNT(*) as cnt
        FROM ecommerce.silver.customers_scd
        WHERE is_current = TRUE
        GROUP BY customer_id
        HAVING cnt > 1
    )
""").collect()[0]["dup_count"]

if duplicate_current == 0:
    print("    Each customer has exactly one current record!")
else:
    print(f"     {duplicate_current} customers have multiple current records!")

SCD TYPE 2 VERIFICATION

📊 Record Counts:
   Total records:      99,441
   Current records:    98,441  (is_current=TRUE)
   Historical records: 1,000 (is_current=FALSE)

📊 Version Distribution:
+-------+------------+-------------+
|version|record_count|current_count|
+-------+------------+-------------+
|      1|       99441|        98441|
+-------+------------+-------------+

 Sample Customer Full History:
+-----------+-------------+--------------+--------------------+------------------+----------+-------+
|customer_id|customer_city|customer_state|effective_start_date|effective_end_date|is_current|version|
+-----------+-------------+--------------+--------------------+------------------+----------+-------+
+-----------+-------------+--------------+--------------------+------------------+----------+-------+

 Data Quality Check — No overlapping date ranges:
    No overlapping date ranges found!
 Data Quality Check — One current record per customer:
    Each customer has exactly one cur

Business Queries Using SCD Type 2

In [0]:
print("=" * 60)
print("BUSINESS QUERIES USING SCD TYPE 2")
print("=" * 60)

# ── Query 1: Current customer locations ──────────────────────────
print("\n  Current customer distribution by state:")
spark.sql("""
    SELECT
        customer_state,
        COUNT(*) as customer_count
    FROM ecommerce.silver.customers_scd
    WHERE is_current = TRUE
    GROUP BY customer_state
    ORDER BY customer_count DESC
    LIMIT 10
""").display()

# ── Query 2: Top cities customers moved TO ───────────────────────
print("  Top cities customers moved TO:")
spark.sql("""
    SELECT
        customer_city       AS moved_to_city,
        customer_state      AS moved_to_state,
        COUNT(*)            AS num_customers_moved_here
    FROM ecommerce.silver.customers_scd
    WHERE is_current = TRUE
    AND version > 1
    GROUP BY customer_city, customer_state
    ORDER BY num_customers_moved_here DESC
    LIMIT 10
""").display()

# ── Query 3: How many versions per customer ──────────────────────
print("  Customer change frequency:")
spark.sql("""
    SELECT
        max_version,
        COUNT(*) AS num_customers
    FROM (
        SELECT
            customer_id,
            MAX(version) AS max_version
        FROM ecommerce.silver.customers_scd
        GROUP BY customer_id
    ) version_summary
    GROUP BY max_version
    ORDER BY max_version
""").display()

# ── Query 4: Point-in-time query ─────────────────────────────────
print("  Where were customers located in 2017?")
spark.sql("""
    SELECT
        customer_state,
        COUNT(*) as customer_count
    FROM ecommerce.silver.customers_scd
    WHERE effective_start_date <= '2017-12-31'
    AND   effective_end_date   >  '2017-01-01'
    GROUP BY customer_state
    ORDER BY customer_count DESC
    LIMIT 10
""").display()

BUSINESS QUERIES USING SCD TYPE 2

  Current customer distribution by state:


customer_state,customer_count
SP,41359
RJ,12719
MG,11491
RS,5397
PR,4996
SC,3607
BA,3344
DF,2115
ES,2015
GO,1998


  Top cities customers moved TO:


moved_to_city,moved_to_state,num_customers_moved_here


  Customer change frequency:


max_version,num_customers
1,99441


  Where were customers located in 2017?


customer_state,customer_count
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033
GO,2020


Add to Gold Layer

In [0]:
# Update Gold layer to use SCD Type 2 data
# New gold table: customer location history analysis

customer_location_analysis = spark.sql("""
    SELECT
        s.customer_state                        AS state,
        COUNT(DISTINCT s.customer_unique_id)    AS total_customers,
        SUM(CASE WHEN s.version > 1 THEN 1 
                 ELSE 0 END)                    AS customers_who_moved,
        ROUND(
            SUM(CASE WHEN s.version > 1 THEN 1 ELSE 0 END) * 100.0
            / COUNT(DISTINCT s.customer_unique_id), 2
        )                                       AS pct_who_moved,
        ROUND(AVG(s.version), 2)                AS avg_versions,
        MAX(s.version)                          AS max_versions,
        current_timestamp()                     AS updated_at
    FROM ecommerce.silver.customers_scd s
    WHERE s.is_current = TRUE
    GROUP BY s.customer_state
    ORDER BY total_customers DESC
""")

(customer_location_analysis.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("ecommerce.gold.customer_location_analysis")
)

count = spark.table("ecommerce.gold.customer_location_analysis").count()
print(f" gold.customer_location_analysis: {count:,} records")
spark.table("ecommerce.gold.customer_location_analysis").display(10)

 gold.customer_location_analysis: 27 records


state,total_customers,customers_who_moved,pct_who_moved,avg_versions,max_versions,updated_at
SP,39938,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
RJ,12260,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
MG,11124,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
RS,5212,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
PR,4834,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
SC,3504,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
BA,3242,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
DF,2051,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
ES,1947,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z
GO,1930,0,0.00,1.0,1,2026-02-24T00:14:01.919157Z


Final Summary

In [0]:
print("=" * 60)
print("SCD TYPE 2 PIPELINE — COMPLETE SUMMARY")
print("=" * 60)

total     = spark.table("ecommerce.silver.customers_scd").count()
current   = spark.sql("""
    SELECT COUNT(*) as c
    FROM ecommerce.silver.customers_scd
    WHERE is_current = TRUE
""").collect()[0]["c"]
historical = total - current
v2_plus   = spark.sql("""
    SELECT COUNT(DISTINCT customer_id) as c
    FROM ecommerce.silver.customers_scd
    WHERE version > 1
""").collect()[0]["c"]

print(f"""
Table:     ecommerce.silver.customers_scd

Records:
  Total:      {total:,}
  Current:    {current:,}   (is_current = TRUE)
  Historical: {historical:,}  (is_current = FALSE)
  Changed:    {v2_plus:,}   (version > 1)

Quality Checks:
   No overlapping date ranges
   One current record per customer
   Version numbers sequential

Gold Table:
   gold.customer_location_analysis created
""")


SCD TYPE 2 PIPELINE — COMPLETE SUMMARY

Table:     ecommerce.silver.customers_scd

Records:
  Total:      99,441
  Current:    98,441   (is_current = TRUE)
  Historical: 1,000  (is_current = FALSE)
  Changed:    0   (version > 1)

Quality Checks:
   No overlapping date ranges
   One current record per customer
   Version numbers sequential

Gold Table:
   gold.customer_location_analysis created

